## EDA

Lisa Fung, 8/3/24

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [62]:
import geopandas as gpd

In [3]:
acs_df = pd.read_csv("../Data/acs_5yr_est_selected_economic_characteristics_2010-2022.csv")
commodities_df = pd.read_csv("../Data/all_commodities_fixed.csv")
stocks_prices_df = pd.read_csv("../Data/all_stock_and_etfs.csv")
meat_store_df = pd.read_csv("../Data/Meat_Stats_Cold_Storage.csv")
meat_prod_df = pd.read_csv("../Data/Meat_Stats_Meat_Production.csv")
meat_scounts_df = pd.read_csv("../Data/Meat_Stats_Slaughter_Counts.csv")
meat_sweights_df = pd.read_csv("../Data/Meat_Stats_Slaughter_Weights.csv") # 1983-2023
health_df = pd.read_csv("../Data/Nutrition_Physical_Activity_and_Obesity_Data.csv")
stocks_info_df = pd.read_csv("../Data/stock_descriptions.csv")

C:\Users\lisaf\AppData\Local\Temp\ipykernel_16720\1116362578.py:8: DtypeWarning: Columns (18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  health_df = pd.read_csv("../Data/Nutrition_Physical_Activity_and_Obesity_Data.csv")


In [36]:
# commodities_df['Commodity'].unique()
# stocks_prices_df['Ticker_Symbol'].unique()
# meat_store_df['Animal'].unique()
# meat_scounts_df['Animal'].unique()
# stocks_info_df

## Health Data: Nutrition_Physical_Activity_and_Obesity_Data
Questions:
- Ages: Grade 9-12, Adults 18 and older
- Separated by: 'LocationAbbr', 'Age(years)', 'Education', 'Gender', 'Grade', 'Income', 'Race/Ethnicity'
- Questions: Physical Activity, Obesity/Overweight, Nutrition (Vegetables, fruits, soda/pop)

In [99]:
health_df['Question'].value_counts()
health_df['Class'].unique()

array(['Physical Activity', 'Obesity / Weight Status', 'Sugar Drinks',
       'Fruits and Vegetables', 'Television Viewing'], dtype=object)

In [90]:
health_df.columns
# health_df['Age(years)'].value_counts()

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'Datasource',
       'Class', 'Topic', 'Question', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Alt', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Low_Confidence_Limit', 'High_Confidence_Limit',
       'Sample_Size', 'Total', 'Age(years)', 'Education', 'Gender', 'Grade',
       'Income', 'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID',
       'QuestionID', 'DataValueTypeID', 'LocationID',
       'StratificationCategory1', 'Stratification1',
       'StratificationCategoryId1', 'StratificationID1'],
      dtype='object')

In [86]:
SOCIO_COLS = ['Age(years)', 'Education', 'Gender', 'Grade', 'Income', 'Race/Ethnicity']
# health_df[health_df['Age(years)'].notna()][['Datasource', 'Question', 'LocationAbbr'] + SOCIO_COLS]['LocationAbbr'].value_counts()

In [40]:
health_df.iloc[0]

YearStart                                                                  2019
YearEnd                                                                    2019
LocationAbbr                                                                 AK
LocationDesc                                                             Alaska
Datasource                              Youth Risk Behavior Surveillance System
Class                                                         Physical Activity
Topic                                              Physical Activity - Behavior
Question                      Percent of students in grades 9-12 who achieve...
Data_Value_Unit                                                             NaN
Data_Value_Type                                                           Value
Data_Value                                                                  9.6
Data_Value_Alt                                                              9.6
Data_Value_Footnote_Symbol              

### Health GIS Visualizations

In [ ]:
# Obesity, Overweight
# Percent of adults aged 18 years and older who have an overweight classification 

health_weight_df = 